# MIDIs to Images Converter (ver. 1.0)

***

Powered by tegridy-tools: https://github.com/asigalov61/tegridy-tools

***

#### Project Los Angeles

#### Tegridy Code 2023

***

# (SETUP ENVIRONMENT)

In [ ]:
#@title Install all dependencies (run only once per session)

!git clone https://github.com/asigalov61/tegridy-tools
!pip install tqdm

In [ ]:
#@title Import all needed modules

print('Loading needed modules. Please wait...')
import os

import math
import statistics
import random

from collections import Counter

from tqdm import tqdm

if not os.path.exists('/content/Dataset'):
    os.makedirs('/content/Dataset')

if not os.path.exists('/content/Images'):
    os.makedirs('/content/Images')

print('Loading TMIDIX module...')
os.chdir('/content/tegridy-tools/tegridy-tools')

import TMIDIX

import matplotlib.pyplot as plt

print('Done!')

os.chdir('/content/')
print('Enjoy! :)')

# (DOWNLOAD AND UNZIP SAMPLE DATASET)

In [ ]:
# @title Download and unzip sample MIDI dataset
%cd /content/Dataset

!wget https://github.com/asigalov61/Tegridy-MIDI-Dataset/raw/master/Tegridy-Children-Songs-CC-BY-NC-SA.zip

!unzip Tegridy-Children-Songs-CC-BY-NC-SA.zip

!rm Tegridy-Children-Songs-CC-BY-NC-SA.zip

%cd /content/

# (FILE LIST)

In [ ]:
#@title Save file list
###########

print('Loading MIDI files...')
print('This may take a while on a large dataset in particular.')

dataset_addr = "/content/Dataset"
# os.chdir(dataset_addr)
filez = list()
for (dirpath, dirnames, filenames) in os.walk(dataset_addr):
    filez += [os.path.join(dirpath, file) for file in filenames]
print('=' * 70)

if filez == []:
    print('Could not find any MIDI files. Please check Dataset dir...')
    print('=' * 70)

print('Randomizing file list...')
random.shuffle(filez)
print('Done!')

# (PROCESS MIDIs)

In [ ]:
#@title Process MIDIs with TMIDIX MIDI processor

print('=' * 70)
print('TMIDIX MIDI Processor')
print('=' * 70)
print('Starting up...')
print('=' * 70)

###########

START_FILE_NUMBER = 0
LAST_SAVED_BATCH_COUNT = 0

input_files_count = START_FILE_NUMBER
files_count = LAST_SAVED_BATCH_COUNT

melody_chords_f = []

stats = [0] * 129

avg_durs = []

avg_times = []

print('Processing MIDI files. Please wait...')
print('=' * 70)

for f in tqdm(filez[START_FILE_NUMBER:]):

  try:

        input_files_count += 1

        fn = os.path.basename(f)
        fn1 = fn.split('.mid')[0]

        # Filtering out giant MIDIs
        file_size = os.path.getsize(f)

        if file_size <= 1000000:

          #=======================================================
          # START PROCESSING

          # Convering MIDI to ms score with MIDI.py module
          score = TMIDIX.midi2single_track_ms_score(open(f, 'rb').read(), recalculate_channels=False)

          # INSTRUMENTS CONVERSION CYCLE
          events_matrix = []
          itrack = 1
          patches = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

          while itrack < len(score):
              for event in score[itrack]:
                  if event[0] == 'note' or event[0] == 'patch_change':
                      events_matrix.append(event)
              itrack += 1

          events_matrix.sort(key=lambda x: x[1])

          events_matrix1 = []

          for event in events_matrix:
                  if event[0] == 'patch_change':
                        patches[event[2]] = event[3]

                  if event[0] == 'note':
                    if event[3] != 9:
                        event.extend([patches[event[3]]])

                        if events_matrix1:
                            if (event[1] == events_matrix1[-1][1]):
                                if ([event[3], event[4]] != events_matrix1[-1][3:5]):
                                    events_matrix1.append(event)
                            else:
                                events_matrix1.append(event)

                        else:
                            events_matrix1.append(event)

          if len(events_matrix1) > 0:
            if min([e[1] for e in events_matrix1]) >= 0 and min([e[2] for e in events_matrix1]) >= 0:

              #=======================================================

              for e in events_matrix1:
                e[1] = int(e[1] / 32)
                e[2] = int(e[2] / 32)

              # Sorting by pitch, then by start-time
              events_matrix1.sort(key=lambda x: x[4], reverse=True)
              events_matrix1.sort(key=lambda x: x[1])

              #=======================================================

              pe = events_matrix1[0]

              times = []

              abs_time = 0

              melody_chords = []

              for e in events_matrix1:

                # Cliping all values...
                time = max(0, min(127, e[1]-pe[1]))
                dur = max(1, min(127, e[2]))
                ptc = max(1, min(127, e[4]))

                cha = max(0, min(15, e[3]))

                pat = max(0, min(127, e[6]))

                times.append(max(0, e[1]-pe[1]))

                abs_time += time

                melody_chords.append([time, dur, ptc, pat])

                pe = e

              melody_chords_f.append([abs_time, melody_chords, fn1])

  except KeyboardInterrupt:
    print('SAVING PROGRESS AND QUITTING...')
    print('=' * 70)
    break

  except Exception as ex:
      print('WARNING !!!')
      print('=' * 70)
      print('Bad MIDI:', f)
      print('Error detected:', ex)
      print('=' * 70)
      continue

print('DONE !!!')
print('=' * 70)

# (CONVERT MIDIs TO IMAGES)

In [ ]:
# @title Convert processed MIDIs to images
image_resolution = 512 # @param {type:"slider", min:128, max:1024, step:128}
MIDI_notes_range = 96 # @param {type:"slider", min:64, max:128, step:8}
timings_density = 32 # @param {type:"slider", min:32, max:128, step:16}

RESOLUTION = image_resolution
RANGE = MIDI_notes_range
TIMINGS = timings_density

print('=' * 70)
print('Converting to images... Please wait...')
print('=' * 70)

for m in tqdm(melody_chords_f):

  try:

    matrix = []

    for i in range(m[0] * TIMINGS):
      matrix.append([0] * RANGE)

    a_time = 0

    for i in range(len(m[1])):
      note = m[1][i]

      a_time += note[0]

      for j in range(note[1]):
        matrix[a_time+j][max(0, min((RANGE, note[2]-(128-RANGE))))] = (note[3] + 1) + 256


    matrix2 = []

    for i in range(RESOLUTION):
      matrix2.append([0] * RESOLUTION)

    for i in range(RESOLUTION // RANGE):
      mat = matrix[(i * RESOLUTION):((i+1) * RESOLUTION)]

      for j in range(len(mat)):

        for k in range(RANGE):
          matrix2[j][k+(i * RANGE)] = mat[j][k]

    x = []
    y = []

    plt.imshow(matrix2)
    plt.savefig('/content/Images/'+m[2]+'.png', dpi=300)
    # plt.show()
    plt.close()

  except KeyboardInterrupt:
    print('SAVING PROGRESS AND QUITTING...')
    print('=' * 70)
    break

  except Exception as ex:
      print('WARNING !!!')
      print('=' * 70)
      print('Bad MIDI:', m[2])
      print('Error detected:', ex)
      print('=' * 70)
      continue

print('Done!')
print('=' * 70)

# Congrats! You did it! :)